In [ ]:
! pip install -qU keras pandas tensorflow

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
 
img_width, img_height = 224, 224

In [ ]:
train_data_dir = 'roasted/train'
validation_data_dir = 'roasted/test'
nb_train_samples = 761 + 745 + 741 + 743
nb_validation_samples = 100
epochs = 10
batch_size = 16


In [ ]:
ds

In [ ]:
!unzip train.zip